In [ ]:
# install dependencies: 
!pip install pyyaml==5.1
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab


In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog


In [ ]:
%matplotlib inline
#The line above is necesary to show Matplotlib's plots inside a Jupyter Notebook
from matplotlib import pyplot as plt
import os

# img_file = "Geel/Geel-002.jpg"
# img_file = "Geel/MarktGeel-3.jpg"
img_file = "Geel/2016 - Kristof Donné - Spelende kinderen Markt-2.jpg"
img_filename, img_file_extension = os.path.splitext(img_file)

im = cv2.imread(img_file)

# !wget http://images.cocodataset.org/val2017/000000439715.jpg -q -O input.jpg
# im = cv2.imread("./input.jpg")
# im = cv2.imread("./juli-kosolapova-oGvDWyYct4Y-unsplash.jpg")
# im = cv2.imread("./eliott-reyna-jCEpN62oWL4-unsplash.jpg")
# im = cv2.imread("./kin-li-UOYkntGPvNE-unsplash.jpg")
# im = cv2.imread("./gabriella-clare-marino-2vgx9eOmH0I-unsplash.jpg")
# im = cv2.imread("./gabriella-clare-marino-ms6tf_QVeSQ-unsplash.jpg")
# im = cv2.imread("./jan-antonin-kolar-LuBJ1GSvBl4-unsplash.jpg")
# im = cv2.imread("./alex-vasey-9UhOKfAtjXA-unsplash.jpg")
# im = cv2.imread("./alex-vasey-oLjUXxYX8lQ-unsplash.jpg")
# im = cv2.imread("./2009-Italie-hires-148.jpg")
# im = cv2.imread("./orlando-allo-qRpOzXWsu3c-unsplash.jpg")
# im = cv2.imread("./allen-taylor-0cmJRW5rOJ8-unsplash.jpg")
# im = cv2.imread("./antonio-sessa-8TWBBmha7gI-unsplash.jpg")


In [ ]:
plt.imshow(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
plt.rcParams["figure.figsize"] = (200,100)
plt.show()

In [ ]:
cfg = get_cfg()
cfg.MODEL.DEVICE='cpu'
# add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
# Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
predictor = DefaultPredictor(cfg)
outputs = predictor(im)

In [ ]:
# print(outputs["instances"].pred_classes.size())
# print(outputs["instances"].pred_boxes)
# print(outputs["instances"].pred_classes)
# print(outputs["instances"].pred_masks.size())
# print(outputs["instances"])
# print(vars(outputs["instances"])['_fields'].keys())

In [ ]:
im.size
im.shape


In [ ]:
from detectron2.utils.visualizer import GenericMask

img = im.copy()

img_zero = np.ones(img.shape,dtype=np.uint8)
img_zero[:,:] = (24,10,10)
alpha_mask = np.ones(img.shape,dtype=np.uint8)*255

for i in range(outputs["instances"].pred_classes.shape[0]) :
    instance_class = outputs["instances"].pred_classes[i]
    if(instance_class == 0) :
        polymasks = GenericMask(outputs["instances"].pred_masks[i].numpy(), img.shape[0], img.shape[1]).polygons
        for polymask in polymasks :
            polymask_pairs = [ [x,y] for x,y in zip(polymask[0::2], polymask[1::2]) ]
            pts = np.array([polymask_pairs], np.int32)
            # directly on the image, no blurring: 
            # cv2.fillPoly(img, pts, color=(0, 0, 0))
            # cv2.polylines(img, pts, isClosed=True, color=(255,255,255), thickness=1, lineType=cv2.LINE_AA)
            
            cv2.polylines(img_zero, pts, isClosed=True, color=(255,255,255), thickness=1, lineType=cv2.LINE_AA)
            cv2.fillPoly(alpha_mask, pts, color=(0, 0, 0))

img_zero_line = img_zero.copy()
img_blur = cv2.GaussianBlur(img_zero, (9,9), 11)
img_blur2 = cv2.GaussianBlur(img_zero, (5,5), 7)
img_people = cv2.add(img_blur, img_blur2)
img_people = cv2.add(img_people, img_zero_line)

# https://learnopencv.com/alpha-blending-using-opencv-cpp-python/
background = img.astype(float)
# background = np.zeros(img.shape,dtype=np.uint8).astype(float)
people = img_people.astype(float)
alpha = alpha_mask.astype(float)/255

people = cv2.multiply(1.0-alpha, people)
background = cv2.multiply(alpha, background)
overlay = np.array(cv2.add(people, background), dtype=np.uint8)

plt.imshow(cv2.cvtColor(overlay, cv2.COLOR_BGR2RGB))
plt.rcParams["figure.figsize"] = (200,100)
plt.show()

In [ ]:
cv2.imwrite(img_filename + "-seg" + img_file_extension, overlay)

In [ ]:
img = im[:, :, ::-1]
v = Visualizer(img, MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
img_block = out.get_image()[:, :, ::-1]

plt.imshow(cv2.cvtColor(img_block, cv2.COLOR_BGR2RGB))
plt.rcParams["figure.figsize"] = (200,100)
# plt.show()

In [ ]:
cv2.imwrite(img_filename + "-block" + img_file_extension, img_block)

In [ ]:
# Inference with a keypoint detection model
cfg = get_cfg()   # get a fresh new config
cfg.MODEL.DEVICE='cpu'
cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7  # set threshold for this model
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml")
predictor = DefaultPredictor(cfg)
outputs = predictor(im)


In [ ]:
im_key = overlay.copy()

if(outputs["instances"].has("pred_boxes")) :
    outputs["instances"].remove("pred_boxes") # don't draw the boxes

v = Visualizer(im_key[:,:,::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.0)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))

overlay_skel = out.get_image()[:, :, ::-1]

plt.imshow(cv2.cvtColor(overlay_skel, cv2.COLOR_BGR2RGB))
plt.rcParams["figure.figsize"] = (300,150)
plt.show()

In [ ]:
cv2.imwrite(img_filename + "-skel" + img_file_extension, overlay_skel)

In [ ]:
im_key = im.copy()
v = Visualizer(im_key[:,:,::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))


In [ ]:
plt.imshow(cv2.cvtColor(out.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB))
plt.rcParams["figure.figsize"] = (300,150)
# plt.show()

In [ ]:
# -------------------------------------------DEKR----------------------------------------------------- #

In [ ]:
dekr_folder="/src/DEKR"
import sys
sys.path.append(dekr_folder+"/lib")
from PIL import Image

import torch
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms


In [ ]:
sys.path.append(dekr_folder+"/tools")
import _init_paths
import models

from config import cfg as cfg_dekr
from config import update_config
from core.inference import get_multi_stage_outputs
from core.inference import aggregate_results
from core.nms import pose_nms
from core.match import match_pose_to_heatmap
from utils.transforms import resize_align_multi_scale
from utils.transforms import get_final_preds
from utils.transforms import get_multi_scale_size
from utils.transforms import up_interpolate

CTX = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


In [ ]:
COCO_KEYPOINT_INDEXES = {
    0: 'nose',
    1: 'left_eye',
    2: 'right_eye',
    3: 'left_ear',
    4: 'right_ear',
    5: 'left_shoulder',
    6: 'right_shoulder',
    7: 'left_elbow',
    8: 'right_elbow',
    9: 'left_wrist',
    10: 'right_wrist',
    11: 'left_hip',
    12: 'right_hip',
    13: 'left_knee',
    14: 'right_knee',
    15: 'left_ankle',
    16: 'right_ankle'
}


CROWDPOSE_KEYPOINT_INDEXES = {
    0: 'left_shoulder',
    1: 'right_shoulder',
    2: 'left_elbow',
    3: 'right_elbow',
    4: 'left_wrist',
    5: 'right_wrist',
    6: 'left_hip',
    7: 'right_hip',
    8: 'left_knee',
    9: 'right_knee',
    10: 'left_ankle',
    11: 'right_ankle',
    12: 'head',
    13: 'neck'
}

In [ ]:
def get_pose_estimation_prediction(cfg, model, image, vis_thre, transforms):
    # size at scale 1.0
    base_size, center, scale = get_multi_scale_size(
        image, cfg.DATASET.INPUT_SIZE, 1.0, 1.0
    )

    with torch.no_grad():
        heatmap_sum = 0
        poses = []

        for scale in sorted(cfg.TEST.SCALE_FACTOR, reverse=True):
            image_resized, center, scale_resized = resize_align_multi_scale(
                image, cfg.DATASET.INPUT_SIZE, scale, 1.0
            )

            image_resized = transforms(image_resized)
            image_resized = image_resized.unsqueeze(0)
            # image_resized = image_resized.unsqueeze(0).cuda()

            heatmap, posemap = get_multi_stage_outputs(
                cfg, model, image_resized, cfg.TEST.FLIP_TEST
            )
            heatmap_sum, poses = aggregate_results(
                cfg, heatmap_sum, poses, heatmap, posemap, scale
            )
        
        heatmap_avg = heatmap_sum/len(cfg.TEST.SCALE_FACTOR)
        poses, scores = pose_nms(cfg, heatmap_avg, poses)

        if len(scores) == 0:
            return []
        else:
            if cfg.TEST.MATCH_HMP:
                poses = match_pose_to_heatmap(cfg, poses, heatmap_avg)

            final_poses = get_final_preds(
                poses, center, scale_resized, base_size
            )

        final_results = []
        for i in range(len(scores)):
            if scores[i] > vis_thre:
                final_results.append(final_poses[i])

        if len(final_results) == 0:
            return []

    return final_results

In [ ]:
import types # for SimpleNamespace

# transformation
pose_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])
# cudnn related setting
cudnn.benchmark = cfg_dekr.CUDNN.BENCHMARK
torch.backends.cudnn.deterministic = cfg_dekr.CUDNN.DETERMINISTIC
torch.backends.cudnn.enabled = cfg_dekr.CUDNN.ENABLED

args = types.SimpleNamespace()
args.cfg = dekr_folder+"/experiments/coco/inference_demo_coco.yaml"
args.opts = ["TEST.MODEL_FILE", dekr_folder+"/model/pose_coco/pose_dekr_hrnetw32_coco.pth"]
args.visthre = '0.3'
# print(args)
update_config(cfg_dekr, args)


# print(cfg_dekr.MODEL.NAME)
# print(cfg_dekr.MODEL.SPEC)
pose_model = models.hrnet_dekr.get_pose_net(cfg_dekr, is_train=False)

pose_model = eval('models.'+cfg_dekr.MODEL.NAME+'.get_pose_net')(
        cfg_dekr, is_train=False
)

if cfg_dekr.TEST.MODEL_FILE:
    print('=> loading model from {}'.format(cfg_dekr.TEST.MODEL_FILE))
    pose_model.load_state_dict(torch.load(
        cfg_dekr.TEST.MODEL_FILE, map_location=CTX), strict=False)
else:
    raise ValueError('expected model defined in config at TEST.MODEL_FILE')


In [ ]:
pose_model.to(CTX)
pose_model.eval()


In [ ]:
image_rgb = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
image_pose = image_rgb.copy()

# The code below doesn't seem to run without GPU. There is a .cuda() call deeper in the DEKR core library.
# The writers of the code clearly didn't test the CPU version :-) 
pose_preds = get_pose_estimation_prediction(cfg_dekr, pose_model, image_pose, args.visthre, transforms=pose_transform)


In [ ]:
# !python3 /src/DEKR/tools/inference_demo.py

!python tools/inference_demo.py --cfg experiments/coco/inference_demo_coco.yaml \
    --videoFile ../multi_people.mp4 \
    --outputDir output \
    --visthre 0.3 \
    TEST.MODEL_FILE model/pose_coco/pose_dekr_hrnetw32.pth
!python tools/inference_demo.py --cfg experiments/crowdpose/inference_demo_crowdpose.yaml \
    --videoFile ../multi_people.mp4 \
    --outputDir output \
    --visthre 0.3 \
    TEST.MODEL_FILE model/pose_crowdpose/pose_dekr_hrnetw32.pth \
